In [328]:
import numpy as np


def newgrid(grid, loc_y, loc_x, offset_y, offset_x, tile, orientation):
    return np.where((np.expand_dims(np.arange(grid.shape[0]),1)==loc_y)
                       &(np.expand_dims(np.arange(grid.shape[1]),0)==loc_x),
                    tile[orientation],
                    np.where((np.expand_dims(np.arange(grid.shape[0]),1)==loc_y+offset_y)
                                 &(np.expand_dims(np.arange(grid.shape[1]),0)==loc_x+offset_x),
                             tile[1-orientation],
                             grid))


def rulecheck(cells, tiles, ruletype, param=None):
    tilenums = np.sort(tiles.reshape(-1))
    known = cells[cells != -1]
    sum_known = np.sum(known)
    n_unknown = np.sum(cells == -1).item()
    if ruletype == 'sum':
        return (np.sum(tilenums[:n_unknown]) <= (param - sum_known) <= np.sum(tilenums[::-1][:n_unknown])).item()
    elif ruletype == 'sum_gt':
        return (np.sum(tilenums[::-1][:n_unknown]) > (param - sum_known)).item()
    elif ruletype == 'sum_lt':
        return (np.sum(tilenums[:n_unknown]) > (param - sum_known)).item()
    elif ruletype == 'all_eq':
        return np.unique(known).shape[0] == 0 or (np.unique(known).shape[0] == 1 and np.sum(tilenums == known[0]) >= n_unknown)
    elif ruletype == 'all_ne':
        return np.unique(known).shape[0] != known.shape[0]
    elif ruletype == 'true':
        return True
    else:
        return False
    

def solve(cellgps, rules, tiles, grid=None, stack=None):
    if isinstance(cellgps, str):
        cellgps = np.array([list(i)for i in cellgps.strip().split('\n')])
        grid = np.where(cellgps == '.', 9, -1)
        stack = np.array([k[1:] 
                          for k in sorted([[c.item(),i,j] 
                                           for i,r in enumerate(cellgps) 
                                           for j,c in enumerate(r) 
                                           if c != '.'])])
    if all([rulecheck(grid.reshape(-1)[cellgps.reshape(-1) == i], 
                      tiles, 
                      *rules[i])
            for i in np.unique(cellgps)[np.unique(cellgps)!='.']]):
        if np.min(grid) > -1:
            return [grid]
        else:
            loc_y, loc_x = stack[0]
            return [s
                   for offset_y, offset_x in [[-1, 0], [0, 1], [1, 0], [0, -1]]
                   for tile in tiles
                   for orientation in range(len({*tile}))
                   for s in solve(cellgps,
                                  rules,
                                  tiles[np.any(tiles != [tile], axis=1)],
                                  newgrid(grid, loc_y, loc_x, offset_y, offset_x, tile, orientation),
                                  stack[np.any(stack != [loc_y, loc_x], axis=1)
                                        &np.any(stack != [loc_y + offset_y, loc_x + offset_x], axis=1)])
                   if grid[loc_y, loc_x] == grid[loc_y + offset_y, loc_x + offset_x] == -1]
    else:
        return []

In [330]:
cellgps = """
......
.ABZC.
.ZBBC.
.BB...
.DE...
.DEFF.
.ZZGG.
......
......
"""

rules = {
    'A': ['sum_gt', 1],
    'B': ['all_eq'],
    'C': ['sum', 10],
    'D': ['sum', 8],
    'E': ['sum', 2],
    'F': ['sum', 7],
    'G': ['sum', 6],
    'Z': ['true'],
}

tiles = np.array([(5, 3), (0, 3), (1, 1), (6, 6), (3, 6), (3, 2), (2, 0), (5, 0), (0, 0), (1, 3)])

solve(cellgps, rules, tiles)

[array([[9, 9, 9, 9, 9, 9],
        [9, 2, 3, 0, 5, 9],
        [9, 0, 3, 3, 5, 9],
        [9, 3, 3, 9, 9, 9],
        [9, 6, 1, 9, 9, 9],
        [9, 2, 1, 1, 6, 9],
        [9, 0, 0, 0, 6, 9],
        [9, 9, 9, 9, 9, 9],
        [9, 9, 9, 9, 9, 9]])]

In [329]:
cellgps = """
......
.ZAAZ.
.ZBBC.
.DBBC.
.DBEC.
.FF...
.FF...
......
"""

rules = {
    'A': ('sum', 9),
    'B': ('sum', 0),
    'C': ('all_eq',),
    'D': ('sum', 6),
    'E': ('sum_gt', 4),
    'F': ('sum', 4),
    'Z': ('true',),
}

tiles = np.array([(2, 0), (5, 3), (0, 3), (1, 1), (6, 0), (1, 3), (4, 0), (0, 1), (2, 3), (3, 3)])

solve(cellgps, rules, tiles)

[array([[9, 9, 9, 9, 9, 9],
        [9, 3, 5, 4, 2, 9],
        [9, 2, 0, 0, 3, 9],
        [9, 3, 0, 0, 3, 9],
        [9, 3, 0, 6, 3, 9],
        [9, 1, 1, 9, 9, 9],
        [9, 1, 1, 9, 9, 9],
        [9, 9, 9, 9, 9, 9]])]